In [2]:
import torch
import pandas as pd
import torch.nn as nn

In [7]:
df = pd.read_csv("train.csv")
dft = pd.read_csv("test.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [9]:
data = df.drop(['PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket'], axis=1)
data
#torch.Tensor(df.loc[1][:-1].to_numpy().astype("float32"))

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,male,22.0,7.2500,S
1,1,1,female,38.0,71.2833,C
2,1,3,female,26.0,7.9250,S
3,1,1,female,35.0,53.1000,S
4,0,3,male,35.0,8.0500,S
...,...,...,...,...,...,...
886,0,2,male,27.0,13.0000,S
887,1,1,female,19.0,30.0000,S
888,0,3,female,NaN,23.4500,S
889,1,1,male,26.0,30.0000,C


In [12]:
#one_hoter = {'Iris-setosa': [[1], [0], [0]], "Iris-virginica": [[0], [0], [1]], "Iris-versicolor": [[0], [1], [0]]}
one_hoter = {'Iris-setosa': [1, 0, 0], "Iris-virginica": [0, 0, 1], "Iris-versicolor": [0, 1, 0]}

In [13]:
from torch.utils.data import Dataset, DataLoader

In [14]:
class IrisDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame):
        self.df = dataframe
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, ind):
        raw = self.df.loc[ind].to_numpy()
        target = raw[-1]
        target = torch.Tensor(one_hoter[target])
        samples = torch.Tensor(raw[:-1].astype("float32"))
        return samples, target

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(4, 32)
        self.f_act = nn.ReLU()
        self.linear2 = nn.Linear(32, 3)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, X):
        #norm = self.norm(X)
        res = self.linear1(X)
        res = self.f_act(res)
        res = self.linear2(res)
        res = self.softmax(res)
        return res

In [21]:
from torchsummary import summary
summary(Net())

Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            160
├─ReLU: 1-2                              --
├─Linear: 1-3                            99
├─Softmax: 1-4                           --
Total params: 259
Trainable params: 259
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            160
├─ReLU: 1-2                              --
├─Linear: 1-3                            99
├─Softmax: 1-4                           --
Total params: 259
Trainable params: 259
Non-trainable params: 0

In [17]:
def train_one_epoch(epoch, trainloader, optimizer, model, criterion, dataset):
    correct = 0
    epoch_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        x, y = data
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        epoch_loss += outputs.shape[0] * loss.item()
        optimizer.step()
        outputs = (outputs > 0.5).float()
        correct += (outputs == y).float().sum()
    print(
        f'TRAIN [{epoch + 1}] loss: {epoch_loss / dataset.__len__():.3f}')

In [18]:
dataset = IrisDataset(df)
trainloader = DataLoader(dataset, batch_size=32,  shuffle=True)
net = Net()
optimizer = torch.optim.RMSprop(params=net.parameters())
loss = nn.CrossEntropyLoss()

In [19]:
for epoch in range(20):
    train_one_epoch(epoch, trainloader, optimizer, net, loss, dataset)

TRAIN [1] loss: 1.044
TRAIN [2] loss: 0.893
TRAIN [3] loss: 0.808
TRAIN [4] loss: 0.762
TRAIN [5] loss: 0.752
TRAIN [6] loss: 0.729
TRAIN [7] loss: 0.718
TRAIN [8] loss: 0.700
TRAIN [9] loss: 0.673
TRAIN [10] loss: 0.661
TRAIN [11] loss: 0.702
TRAIN [12] loss: 0.644
TRAIN [13] loss: 0.650
TRAIN [14] loss: 0.674
TRAIN [15] loss: 0.615
TRAIN [16] loss: 0.641
TRAIN [17] loss: 0.624
TRAIN [18] loss: 0.619
TRAIN [19] loss: 0.617
TRAIN [20] loss: 0.614


In [20]:
net(torch.Tensor(df.loc[1][:-1].to_numpy().astype("float32"))[None, :])

tensor([[9.9290e-01, 7.0958e-03, 1.4098e-09]], grad_fn=<SoftmaxBackward0>)

In [24]:
net.eval()

Net(
  (linear1): Linear(in_features=4, out_features=32, bias=True)
  (f_act): ReLU()
  (linear2): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)

In [ ]:
net(data)